# Lab2_2024

## 实验⽬标
- 掌握数值线性代数的⼀般算法：LU 分解、SOR
- 学会导⼊并使⽤数值计算库：numpy、scipy、matplotlib
- ⽣成实验⽤的数据和函数
- 学会使⽤线性代数建模并求解问题

## 实验方法
- 安装、调试python环境
- 生成线性方程组
- 使用SOR迭代法求解线性方程组

## 实验实现

In [2]:
import numpy as np
def generate_matrix(school_number):
    class_code = int(str(school_number)[-4:-2])
    individual_code = int(str(school_number)[-2:])
    n = class_code
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                A[i, j] = individual_code
            else:
                A[i, j] = (individual_code % (j+1)) / (n+1)
    b = np.array([(i + 1 + individual_code) * 2 for i in range(n)])
    return A, b

In [3]:
school_number = 23030320 # 换成你的学号
A, b = generate_matrix(school_number)
print("A:\n", A)
print("b:\n", b)

A:
 [[20.   0.   0.5]
 [ 0.  20.   0.5]
 [ 0.   0.  20. ]]
b:
 [42 44 46]


In [4]:
# from scipy.linalg import solve
# # 解线性⽅程组Ax = b
# x = solve(A, b)
# print(x) # 输出解向量

In [7]:
def doolittle_decomposition(A):
    """
    Perform Doolittle LU decomposition on the matrix A.
    
    Parameters:
    A (np.array): The input square matrix to be decomposed.
    
    Returns:
    L (np.array): Lower triangular matrix with ones on the diagonal.
    U (np.array): Upper triangular matrix.
    """
    n = A.shape[0]
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(n):
        # L is a unit lower triangular matrix, so we set the diagonal elements to 1
        L[i, i] = 1.0
        
        # Calculate U's elements
        for k in range(i, n):
            s1 = sum(L[i, j] * U[j, k] for j in range(i))
            U[i, k] = A[i, k] - s1

        # Calculate L's elements
        for k in range(i + 1, n):
            s2 = sum(L[k, j] * U[j, i] for j in range(i))
            L[k, i] = (A[k, i] - s2) / U[i, i]

    return L, U

def solve_linear_system(L, U, b):
    """
    Solve the linear system Ax = b using the precomputed L and U matrices.
    
    Parameters:
    L (np.array): The lower triangular matrix from LU decomposition.
    U (np.array): The upper triangular matrix from LU decomposition.
    b (np.array): The right-hand side vector of the equation.
    
    Returns:
    x (np.array): The solution vector.
    """
    n = L.shape[0]
    y = np.zeros(n)
    x = np.zeros(n)

    # Solve Ly = b for y
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]

    # Solve Ux = y for x
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

    return x

# Decompose matrix A into L and U
L, U = doolittle_decomposition(A)

# Solve the linear system
solution = solve_linear_system(L, U, b)

print("Solution:", solution)

Solution: [2.0425 2.1425 2.3   ]


In [5]:
def sor(A, b, omega, initial_guess, tolerance=1e-10, max_iterations=1000):
    n = len(b)
    x = initial_guess.copy()
    for iteration in range(max_iterations):
        x_new = np.zeros_like(x)
        for i in range(n):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
            x_new[i] = (1 - omega) * x[i] + omega * x_new[i]
        
        if np.linalg.norm(x_new - x) < tolerance:
            return x_new, iteration
        
        x = x_new
    
    print("达到最⼤迭代次数，失败")
    return x, max_iterations

In [6]:
initial_guess = np.zeros(3)
omega = 1.5
solution, iterations = sor(A, b, omega, initial_guess)

print(f"解决方案: {solution}")
print(f"迭代次数: {iterations}")

解决方案: [2.0425 2.1425 2.3   ]
迭代次数: 38


In [ ]:
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import WLS
import statsmodels.api as sm
# 权重通常是根据数据点的可靠性来确定的，这里我们简单地假设靠近中间的数据点更可靠
weights = 1 / (1 + np.abs(x - np.median(x)))

# 准备数据进行WLS
X = sm.add_constant(x)  # 添加截距项
wls_model = WLS(y_noisy, X, weights=weights)
results = wls_model.fit()

# 获取拟合结果
y_pred = results.predict(X)

# 绘图
plt.figure(figsize=(10, 6))
plt.scatter(x, y_noisy, label='Noisy data', color='blue')
plt.plot(x, y_true, label='True relationship', linestyle='--', color='green')
plt.plot(x, y_pred, label='WLS fit', color='red')
plt.title('Weighted Least Squares Regression')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## 实验总结
- 迭代法相比较直接法，虽然不能直接通过⼀系列精确的算术运算找到⽅程组的精确解，但是在处理⼤型稀疏矩阵，不要求精确解的情况下更为有效。